In [18]:
import transformers
from transformers import AutoTokenizer
from datasets import DatasetDict, load_from_disk, load_metric
from arguments import DataTrainingArguments, ModelArguments
import numpy as np
datasets = load_from_disk("../data/train_dataset")
print(datasets)

tokenizer = AutoTokenizer.from_pretrained("uomnf97/klue-roberta-finetuned-korquad-v2", use_fast=True)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


In [6]:
question = datasets["train"]["question"]
context = datasets["train"]["context"]

In [74]:
max_seq_length = 2500 # 겹치는 부분이 없도록, 데이터 확인을 위해서 크게 잡음
doc_stride = 128
tokenized_examples = tokenizer(
            question,
            context,
            truncation="only_second",
            max_length=max_seq_length,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            return_token_type_ids=False, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length"
        )

In [67]:
cls_token_id = tokenizer.cls_token_id
sep_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id
unk_token_id = tokenizer.unk_token_id
mask_token_id = tokenizer.mask_token_id

print(f"[CLS] token ID: {cls_token_id}")
print(f"[SEP] token ID: {sep_token_id}")
print(f"[PAD] token ID: {pad_token_id}")
print(f"[UNK] token ID: {unk_token_id}")
print(f"[MASK] token ID: {mask_token_id}")

[CLS] token ID: 0
[SEP] token ID: 2
[PAD] token ID: 1
[UNK] token ID: 3
[MASK] token ID: 4


In [69]:
# Decoding
example = tokenized_examples["input_ids"][3]
decoded_text = tokenizer.decode(example, skip_special_tokens=True)
decoded_text

'11 ~ 12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요? 불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든 것을 불감 ( 佛 ) 이라고 한다. 불감은 그 안에 모신 불상의 양식뿐만 아니라, 당시의 건축 양식을 함께 살필 수 있는 중요한 자료가 된다. n n이 작품은 높이 18cm의 작은 불감으로, 청동으로 불감과 불상을 만들고 그 위에 금칠을 하였다. 불감 내부를 살펴보면 난간을 두른 사각형의 기단 위에 본존불과 양 옆에 보살상이 있으며, 그 위에 기둥과 지붕으로 된 뚜껑이 덮혀 있다. 법당 모양의 뚜껑에는 앞면과 양쪽에 커다란 창문이 있어서 안에 모셔진 불상을 잘 볼 수 있도록 하였다. n n본존불은 얼굴이 추상적이고, 양 어깨를 감싸고 있는 옷은 주름을 간략한 선으로 표현했다. 몸 뒤편에 있는 광배 ( 光 ) 는 머리광배와 몸광배로 나누어져 있으며, 불꽃무늬로 가장자리를 장식하고 있다. 본존불 양 옆의 보살상도 구슬로 장식된 관 ( ) 을 쓰고 있다는 점을 제외하면 형식이나 표현 수법이 본존불과 유사하다. n n불감은 지금도 금색이 찬란하고 지붕에 녹청색이 남아 있는 등 전체적인 보존 상태가 양호하다. 본존불의 긴 허리, 불규칙하게 나타나는 옷주름, 그리고 보살이 쓰고 있는 구슬로 장식한 관 ( ) 등 여러 양식으로 보아 만든 시기는 중국 북방 계통의 영향을 받은 11∼12세기 경으로 추정된다. 이 작품은 고려시대 또는 그 이전의 목조건축 양식과 조각수법을 보여주는 귀중한 예라는 점에서 가치가 크다고 할 수 있다.'

In [70]:
np.array(tokenized_examples["input_ids"]).shape

(3952, 2500)

### 대응되는 토큰들 확인

Special Tokens
* [CLS] token ID: 0
* [SEP] token ID: 2
* [PAD] token ID: 1
* [UNK] token ID: 3
* [MASK] token ID: 4

In [71]:
def convert_input_ids_to_tokens(tokenizer, tokenized_examples):
    tokenized_data = []
    for input_id_list in tokenized_examples["input_ids"]:
        # input_ids 리스트를 다시 토큰으로 변환
        tokens = tokenizer.convert_ids_to_tokens(input_id_list)
        tokenized_data.append(tokens)
    
    return tokenized_data

In [72]:
tokenized_data = np.array(convert_input_ids_to_tokens(tokenizer, tokenized_examples=tokenized_examples))

In [73]:
print(tokenized_examples["input_ids"][1])

[0, 3845, 2125, 4004, 2446, 2084, 7604, 2079, 25087, 2052, 860, 1644, 2073, 35, 2, 11, 5496, 2125, 11984, 11, 4013, 11, 6725, 2125, 11984, 11, 3604, 3845, 2125, 11984, 6233, 4561, 2496, 2259, 4342, 2259, 9746, 2440, 2104, 28674, 18, 22, 2232, 3665, 2104, 2165, 2069, 4372, 2088, 16, 26, 18, 3912, 23249, 2079, 4342, 2200, 4227, 2073, 7087, 6968, 2170, 4154, 19521, 16, 4227, 12617, 2079, 8400, 2116, 4751, 7488, 6815, 16, 4501, 16, 9583, 7606, 27135, 10035, 2226, 2116, 2116, 4605, 2496, 2259, 4342, 2507, 2088, 16, 3666, 2073, 4175, 3719, 3674, 2125, 3908, 2170, 3646, 2052, 6670, 2069, 3645, 1889, 2414, 4342, 2507, 2062, 18, 3727, 19011, 2440, 11358, 4077, 2506, 2079, 9698, 170, 3939, 2079, 3966, 171, 793, 3845, 2125, 3939, 2079, 3872, 2069, 4277, 7488, 2112, 16, 4078, 5496, 2125, 4004, 2446, 2084, 7604, 2138, 11750, 3845, 2125, 4004, 2446, 2084, 7604, 2079, 18773, 2052, 3622, 18, 4077, 2506, 2259, 7097, 2079, 4008, 2069, 3986, 2205, 2307, 3939, 2052, 30142, 2104, 3841, 2079, 4145, 2052, 21

UNK 토큰 개수 확인 (Question, Context) - Train dataset 

In [84]:
def check_unknown(tokenized_samples):
    # tokenized_samples = ["input_ids"] 리스트들
    unk_count_per_index = []
    for i,sample in enumerate(tokenized_samples):
        sample_unk_count = 0
        for token in sample:
            if token == 3: # unk token ids
                sample_unk_count += 1
        
        unk_count_per_index.append([i, sample_unk_count])

    return np.array(unk_count_per_index)

In [85]:
unk_token = check_unknown(tokenized_examples["input_ids"]) # [질문,지문]별 UNK 개수

In [95]:
# 전체 context에 대한 UNK Token
print(np.sum(unk_token[:,1],axis=0))

28343


Unknown token외에 문제점이 보이는 token 확인

In [133]:
special_tokens = ["[CLS]","[PAD]","[SEP]","[MASK]"]
def get_encoded_data(data, index):
    tokens = []
    for i,token in enumerate(data[index]):
        if token in special_tokens:
            continue
        tokens.append(token)
    return tokens

In [172]:
special_tokens = ["[CLS]","[PAD]","[SEP]","[MASK]"] 
print(datasets["train"]["question"][0], datasets["train"]["context"][0])
print(",".join(get_encoded_data(tokenized_data, 0)))
print("number of UNK token :", unk_token[0]) # 총 4개

대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은? 미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\n\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05
대통령,##을,포함,##한,미국,##의,행정부,견제,##권,##을,갖,##는,국가,기관,##은,?,미국,상의,##원,또는,미국,상원,(,United,State,##s,Se,##n,##at

일본어, 한자 확인

In [173]:
from pprint import pprint
# 예시 데이터
pprint(datasets["train"]["question"][4].split("."))
pprint(datasets["train"]["context"][4].split("."))
print("----Tokenize----")
pprint(",".join(get_encoded_data(tokenized_data, 4)))

['명문이 적힌 유물을 구성하는 그릇의 총 개수는?']
['동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기의 유물로, 경상남도 울주군 웅촌면 대복리에서 '
 '출토되었다고 전한다',
 ' 외함(外函)은 청화명문이 있는 백자이며, 그 안쪽에 납작한 금속제 원형 합 2점과 금속제 원통형 합 등 3점의 그릇이 봉안되어 있다',
 '\\n\\n바깥쪽의 외함인 백자 합 동체 중앙부 표면에 청화안료로 쓴 “癸巳二月日 施主承表 兩主”라는 명문이 세로로 세 줄에 걸쳐서 쓰여 '
 '있어 조선 후기인 계사년에 시주자인 승표 부부가 발원하여 만든 것임을 알 수 있다',
 '\\n\\n동아대학교박물관의 계사명 사리구는 정확한 제작연대는 알 수 없지만 명문 등을 통해 적어도 17세기 이후에 제작된 것으로 '
 '추정되는 작품으로, 명문이 있는 조선 후기 경상도 지역 출토 사리장엄구라는 점에서 중요한 가치를 지닌 작품으로 판단된다',
 '\\n\\n조선 후기 사리장엄구는 아직까지 조사와 연구가 거의 이루어지지 않았으나, 이처럼 세트를 갖추어 출토된 유물은 비교적 드문 '
 '편임을 고려할 때, 이 계사명 사리장엄구는 제작연대와 발원자의 이름이 밝혀져 있으며, 지금까지 출토된 예가 드문 비교적 완전한 세트를 '
 '가진 유물이라는 점에서 조선 후기 사리장엄구 연구에 자료적 가치를 지닌 유물이다',
 '']
----Tokenize----
'명문,##이,적힌,유물,##을,구성,##하,##는,그릇,##의,총,개수,##는,?,동아,##대,##학교,##박,##물,##관,##에서,소장,##하고,있,##는,계,##사,##명,사리,##구,##는,총,4,##개,##의,용기,##로,구성,##된,조선,##후,##기,##의,유물,##로,,,경상남도,울주군,웅,##촌,##면,대,##복,##리에,##서,출토,##되,##었,##다고,전한다,.,외,##함,(,外,[UNK],),은,청,##화,##명,##문,##이,있,##는,백자,##이,##며,,,그,안쪽,##에,납작,##한,금속

In [174]:
# 해당 예시에 대한 unk 개수
unk_token[4]

array([ 4, 12])

한자를 대부분 잘 못알아 보는 것을 확인할 수 있다. 그렇다면 이 한자를 어떻게 해야하는가..? Answer에 한자가 포함된 경우를 보자

In [175]:
datasets["train"]["answers"][136]

{'answer_start': [202], 'text': ['‘전의식’(前意識)']}

In [176]:
from pprint import pprint
# 예시 데이터
pprint(datasets["train"]["question"][136].split("."))
pprint(datasets["train"]["context"][136].split("."))
print("----Tokenize----")
pprint(",".join(get_encoded_data(tokenized_data, 136)))

['프로이트의 정신 체계에서 현실적인 정신체계와 본능적 충동의 체계를 제외한 것의 명칭은?']
['프로이트는 꿈을 해석해서 참다운 꿈 생각을 밝혀낼 수 있다고 믿었다',
 ' 또한 환자와 허물없는 대화를 나누는 자유연상법에 따라 환자의 말을 해석함으로써 환자를 정신적 억압으로부터 해방시키고 치료할 수 있다고 '
 '믿었다',
 ' 반복 강박을 비롯한 삶의 충동과 죽음의 충동에 대한 예리한 통찰을 바로 이 책을 통해 엿볼 수 있다',
 '\\n\\n우선 프로이트는 정신의 체계를 ‘의식’, ‘전의식’(前意識), ‘무의식’의 세 가지로 구분하여 보았다',
 ' 의식은 원래 의식된 것으로서 이성적, 합리적, 현실적인 정신의 체계에 해당한다',
 ' 무의식은 의식되지 않은 것으로서 정신 과정의 대부분을 차지하는 무의식적인 본능적 충동의 체계다',
 ' 그런가 하면 전의식은 의식되기 이전의 정신 체계로서 무의식을 걸러서 의식 쪽으로 보내는 역할, 곧 검열을 행하는 정신의 체계다',
 '\\n\\n≪꿈의 해석≫에서 프로이트는 ‘의식’, ‘전의식’, ‘무의식’에 관해서 아직 철저하게 해명하지 못하고 있다',
 ' 따라서 그는 이 책 아울러 1923년에 출판한 ≪자아와 이드≫에서 정신 과정을 보다 더 명확하고 철저하게 밝힌다',
 ' 이 책에서 프로이트는 ‘의식’, ‘전의식’, ‘무의식’을 하나의 의식이라고 말한다',
 ' 곧 의식의 가장 많은 부분을 무의식이 차지하고 있고 가장 적은 부분을 전의식이 차지하고 있으며 이성적 현실 의식 역시 부분적이라는 '
 '것이다',
 ' 그런가 하면 ≪자아와 이드≫에서 프로이트는 정신 과정을 ‘원초아’, ‘자아’, ‘초자아’로 구분하는데 이러한 구분은 이 책에서의 정신 '
 '과정을 한층 더 역동적으로 밝히고 있다',
 ' 본능 충동으로서의 원초아와 도덕 및 양심에 관계되는 초자아는 무의식에 해당하고 현실적 이성 활동은 자아에 속한다',
 ' ≪자아와 이드≫에서 프로이트는 에로스와 타나토스, 곧 ‘사랑의 충동’과 ‘죽음의 충동’을 대립시키

위 데이터를 보면 확실히 한자에 대한 tokenizing이 제대로 이루어지지 않음을 볼 수 있다.

이렇다면 생각나는 문제점은.. 학습이 제대로 되지않아 만약에 한자가 또 들어오면 정답을 제대로 못낼 수 있다는 단점이 있다..

오히려 특수문자, "<< >>"는 제대로 Tokenizing하는 것을 볼 수 있음

만약에 답이 특수문자가 되는 경우에는 어떻게 나오는가?

In [178]:
from pprint import pprint
# 예시 데이터
pprint(datasets["train"]["question"][183].split("."))
pprint(datasets["train"]["context"][183].split("."))
print("----Tokenize----")
pprint(", ".join(get_encoded_data(tokenized_data, 183)))

['천연두에 감염된 모든 사례 중 출혈성 천연두는 얼마의 비중을 차지하는가?']
['출혈성 천연두는 피부, 점막, 소화관에 광범한 내출혈을 동반하는 심각한 천연두 유형이다',
 ' 출혈성 천연두는 전체 천연두 감염 사례의 약 2% 정도였으며, 대부분 성인에게 발병했다 출혈성 천연두의 발진은 수포를 형성하지 않고 '
 '부드러운 상태로 남아 있다',
 ' 대신 피부 아래로 출혈이 일어나서 마치 새까맣게 탄 것처럼 검은색을 띤다 그래서 이 유형의 천연두를 검은마마(black pox)라고도 '
 '했다\\n\\n감염 초기, 또는 전격형(fulminating) 출혈성 천연두는, 감염 이틀째 내지 사흘째가 되면 결막하 출혈로 인해 눈의 '
 '흰자위가 심홍색이 되면서 출혈이 확인된다',
 ' 출혈성 천연두는 어스름한 홍반, 점상출혈 및 비장, 신장, 장막, 근육에 출혈을 일으키고, 드물지만 외심막, 간, 고환, 난소, '
 '방광에도 출혈을 일으킨다',
 ' 환자는 보통 감염 5 ~ 7일째 경미해 보이는 피부 발진만을 남기고 갑자기 죽는다',
 ' 전격형 출혈성 천연두는 8 ~ 10일 정도 생존한 환자들에게서 나타났다',
 ' 발진 초기에 출혈이 나타나며, 발진은 도드라지지 않고 평평하고 수포 단계 이상 발달하지 않는다 출혈성 천연두 초기 환자는 혈중 '
 '응고인자(e',
 'g',
 ' 혈소판, 프로트롬빈, 글로불린)가 감소하고 안티트롬빈이 증가한다',
 ' 말기 환자는 확연한 혈소판감소증을 나타내지만 응고인자 감소는 덜 심하다',
 ' 말기 단계에서는 안티트롬빈 증가도 나타났다 천연두 유형의 병독성에 따라 이 유형의 천연두는 치명적 천연두 감염 사례의 3 ~ 25% '
 '정도로 잡힌다 출혈성 천연두는 대개 사망에 이를 정도로 치명적이다']
----Tokenize----
('천연, ##두, ##에, 감염, ##된, 모든, 사례, 중, 출혈, ##성, 천연, ##두, ##는, 얼마, ##의, 비중, ##을, '
 '차지, ##하, ##는, ##가, ?, 출혈, ##

%나 ()과 같은 특수문자에 대해서는 크게 문제가 되지 않을 것 같음

### \n 개행문자 개수 확인

위 예시를 보았을 때 "\"에 해당하는 부분이 [UNK]로 인식하여, "\\\n"을 개행토큰으로 보지 못한다.  

그렇다면 각 데이터마다 "\n"의 개수는 몇개인가?

In [189]:
def get_newline(data_samples):
    all_lines = []
    for i,data in enumerate(data_samples):
        new_line_count = data.count("\\n")
        all_lines.append([i, new_line_count])
    return all_lines


In [193]:
newline_counts = np.array(get_newline(datasets["train"]["context"]))

In [196]:
print(np.sum(newline_counts[:,1],axis=0))

22808


In [200]:
newline_counts

array([[   0,    4],
       [   1,    2],
       [   2,    5],
       ...,
       [3949,   16],
       [3950,   14],
       [3951,    6]])